# Supply Chain KPI Calculation

Objective:
Compute and analyze key performance indicators (KPIs) to assess supply chain health and planning risks related to inventory, lead times, costs, and quality.

In [2]:
import pandas as pd
import numpy as np

### KPI Dataset Preparation

A working copy of the dataset is created to compute and store KPI-related variables while preserving the original data.


In [3]:
df=pd.read_csv("../Database/supply_chain_data.csv")
kpi_df = df.copy()

### KPI 1 — Stock Coverage Ratio

This KPI measures how well current stock levels can satisfy observed demand.  
Low values indicate potential stock-out risk, while high values indicate sufficient or excess stock.


In [4]:
kpi_df['stock_coverage_ratio'] = kpi_df['Stock levels'] / kpi_df['Number of products sold']
kpi_df['stock_coverage_ratio'].describe()


count    100.000000
mean       0.216744
std        0.422485
min        0.000000
25%        0.048709
50%        0.114958
75%        0.218610
max        3.875000
Name: stock_coverage_ratio, dtype: float64

**Stock Coverage Ratio Results**

- The mean stock coverage ratio is 0.22, indicating that most products have stock levels covering only a small fraction of observed demand.
- The minimum value is 0, confirming that some products are out of stock.
- There is high variability, with a few products slightly overstocked (max = 3.88), but the majority are understocked.
- Overall, the supply chain appears to be under pressure and may require replenishment prioritization.


### KPI 2 — Stock-Out Flag

This binary KPI identifies products that are currently out of stock.  
- 1 indicates stock-out  
- 0 indicates stock is available

It provides a clear, actionable measure of inventory risk.


In [5]:
kpi_df['stock_out_flag'] = np.where(kpi_df['Stock levels'] == 0, 1, 0)
kpi_df['stock_out_flag'].value_counts()

stock_out_flag
0    99
1     1
Name: count, dtype: int64

**Stock-Out Flag Results**

- Out of 100 products, only 1 product is currently out of stock.
- This binary KPI highlights immediate inventory risk for the specific SKU.
- It complements the Stock Coverage Ratio by pinpointing which product needs urgent replenishment.


### KPI 3 — Demand Pressure Index

This KPI captures demand intensity relative to available stock.  
- Higher values indicate stronger demand pressure and potential stock-out risk.
- It allows prioritization of SKUs that need urgent attention to avoid inventory shortages.


In [6]:
kpi_df['demand_pressure_index'] = kpi_df['Number of products sold'] / (kpi_df['Stock levels'] + 1)
kpi_df['demand_pressure_index'].describe()

count    100.000000
mean      24.474793
std       45.195561
min        0.255319
25%        4.354545
50%        8.569780
75%       19.942320
max      301.000000
Name: demand_pressure_index, dtype: float64

**Demand Pressure Index Results**

- The mean demand pressure index is 24.47, indicating that on average, products face high demand relative to stock levels.
- The maximum value of 301 shows that one product is extremely understocked and needs urgent attention.
- There is significant variability across products, allowing prioritization of SKUs at risk.
- Combined with the Stock Coverage Ratio and Stock-Out Flag, this KPI provides a clear picture of inventory pressure across the supply chain.

### KPI 4 — Average Total Lead Time

This KPI measures the overall time required to replenish products across the supply chain.  
- Higher values indicate slower replenishment and higher planning risk.
- It helps identify products that may cause delays in meeting demand.

In [7]:
kpi_df['avg_total_lead_time'] = (
    kpi_df['Lead times'] + kpi_df['Manufacturing lead time'] + kpi_df['Shipping times']
) / 3
kpi_df['avg_total_lead_time'].describe()

count    100.000000
mean      12.160000
std        4.237209
min        3.666667
25%        9.000000
50%       12.333333
75%       14.750000
max       21.000000
Name: avg_total_lead_time, dtype: float64

**Average Total Lead Time Results**

- The mean average total lead time is 12.16 days, indicating a moderate replenishment cycle for most products.
- The minimum value is 3.67 days, while the maximum is 21 days, showing variability across SKUs.
- Products with long lead times may pose planning risks and should be monitored closely.


### KPI 5 — Manufacturing Lead Time Share

This KPI measures the proportion of total lead time attributable to manufacturing activities.
- Higher values indicate that production is the main source of delays.
- Lower values suggest that supplier or shipping delays dominate.
- It helps prioritize process improvements where bottlenecks exist.


In [8]:
kpi_df['mfg_lead_time_share'] = kpi_df['Manufacturing lead time'] / kpi_df['avg_total_lead_time']
kpi_df['mfg_lead_time_share'].describe()

count    100.000000
mean       1.179651
std        0.580249
min        0.103448
25%        0.744318
50%        1.146429
75%        1.580437
max        2.545455
Name: mfg_lead_time_share, dtype: float64

**Manufacturing Lead Time Share Results**

- The median value is 1.15, indicating that for more than half of the products, manufacturing is the primary contributor to total lead time.
- Values above 1 highlight products where production dominates replenishment delays.
- This KPI helps identify manufacturing bottlenecks and prioritize process improvements.


### KPI 6 — Long Lead Time Flag

This binary KPI identifies products with above-median total lead times.
- 1 indicates products at risk of delayed replenishment.
- 0 indicates normal lead time.
- It helps prioritize SKUs for planning and supply chain interventions.


In [9]:
median_lead_time = kpi_df['avg_total_lead_time'].median()
kpi_df['long_lead_time_flag'] = np.where(kpi_df['avg_total_lead_time'] > median_lead_time, 1, 0)
kpi_df['long_lead_time_flag'].value_counts()


long_lead_time_flag
0    55
1    45
Name: count, dtype: int64

**Long Lead Time Flag Results**

- 45 products have above-median total lead times, indicating potential planning risks.
- 55 products have normal lead times.
- This KPI helps focus attention on SKUs that may delay supply chain operations.


### KPI 7 — Cost per Unit Sold

This KPI measures overall cost efficiency relative to the number of units sold.
- Higher values indicate lower efficiency and higher supply chain cost per product.
- Lower values indicate better cost management.
- It helps identify products where supply chain costs are disproportionately high.


In [10]:
kpi_df['cost_per_unit_sold'] = (
    kpi_df['Manufacturing costs'] + kpi_df['Shipping costs']
) / kpi_df['Number of products sold']
kpi_df['cost_per_unit_sold'].describe()

count    100.000000
mean       0.298431
std        0.645369
min        0.007683
25%        0.060763
50%        0.105294
75%        0.292834
max        4.842812
Name: cost_per_unit_sold, dtype: float64

**Cost per Unit Sold Results**

- The mean cost per unit sold is 0.30, with a high standard deviation of 0.65.
- The maximum value of 4.84 highlights a few products that are very expensive to produce and ship.
- This KPI identifies products with potentially inefficient cost structures for further analysis.


### KPI 8 — Shipping Cost Intensity

This KPI measures logistics efficiency by comparing shipping costs to units sold.
- Higher values indicate disproportionate shipping costs for certain SKUs.
- It helps identify opportunities to optimize transportation and reduce costs.


In [11]:
kpi_df['shipping_cost_intensity'] = kpi_df['Shipping costs'] / kpi_df['Number of products sold']
kpi_df['shipping_cost_intensity'].describe()

count    100.000000
mean       0.037743
std        0.109972
min        0.001544
25%        0.007025
50%        0.013382
75%        0.027101
max        1.006810
Name: shipping_cost_intensity, dtype: float64

**Shipping Cost Intensity Results**

- The mean shipping cost per unit sold is 0.038, with a standard deviation of 0.11.
- The maximum value of 1.01 shows that some products have disproportionately high shipping costs.
- This KPI helps identify SKUs where transportation efficiency can be improved.


### KPI 9 — Total Cost per Revenue

This KPI measures overall cost efficiency relative to revenue generated.  
- Higher values indicate products where costs consume a larger portion of revenue.
- It helps identify low-margin or potentially unprofitable SKUs.


In [12]:
kpi_df['total_cost_per_revenue'] = (kpi_df['Manufacturing costs'] + kpi_df['Shipping costs']) / kpi_df['Revenue generated']
kpi_df['total_cost_per_revenue'].describe()

count    100.000000
mean       0.013774
std        0.013877
min        0.000635
25%        0.005137
50%        0.009072
75%        0.018297
max        0.061861
Name: total_cost_per_revenue, dtype: float64

**Total Cost per Revenue Results**

- The mean total cost per revenue is 0.0138, showing that on average costs consume a very small portion of revenue.
- The maximum value of 0.0619 highlights products with higher relative costs, which may require efficiency improvements.
- This KPI, together with Cost per Unit Sold and Shipping Cost Intensity, provides a clear view of supply chain cost performance.


### KPI — Quality & Risk KPIs

Quality-related KPIs are critical to assess supply chain reliability and operational risk.
They help identify products that may cause rework, delays, or additional costs due to quality failures.


In [13]:
df['Inspection results'].value_counts()


Inspection results
Pending    41
Fail       36
Pass       23
Name: count, dtype: int64

### KPI 10 — Inspection Failure Rate

The inspection failure rate measures the proportion of products that fail quality inspections.

- Only completed inspections (Pass + Fail) are considered.
- Pending inspections are excluded to avoid bias.
- A high failure rate indicates elevated quality and operational risk.

This KPI highlights products that may require rework, additional controls, or supplier intervention.


In [14]:
inspection_counts = df['Inspection results'].value_counts()

failed = inspection_counts.get('Fail', 0)
passed = inspection_counts.get('Pass', 0)

inspection_failure_rate = failed / (failed + passed)

inspection_failure_rate


np.float64(0.6101694915254238)

**Inspection Failure Rate Result**

The inspection failure rate is 61.02%, meaning that more than half of inspected products fail quality checks.

This indicates a significant quality risk within the supply chain and suggests the need for:
- Improved quality controls
- Supplier or manufacturing process audits
- Preventive quality assurance measures


### KPI 11 — Defect Rate

The defect rate measures the proportion of defective units for each product.
It reflects the severity of quality issues and complements inspection-based KPIs.

Higher defect rates indicate increased rework, waste, and operational risk.


In [15]:
df['Defect rates'].describe()


count    100.000000
mean       2.277158
std        1.461366
min        0.018608
25%        1.009650
50%        2.141863
75%        3.563995
max        4.939255
Name: Defect rates, dtype: float64

**Defect Rate Results**

- The average defect rate is 2.28%, with a median of 2.14%.
- The top 25% of products have defect rates above 3.56%, indicating elevated quality risk.
- Maximum defect rates approach 5%, suggesting severe quality issues for certain products.

This KPI highlights the severity of quality problems and their potential impact on cost, lead time, and service level.


### KPI 12 — Quality Cost Exposure

This KPI estimates the financial exposure caused by quality issues by combining defect rates, manufacturing costs, and revenue.

Higher values indicate products where quality problems have a significant financial impact.


In [19]:
kpi_df['quality_cost_exposure'] = (
    kpi_df['Defect rates'] *
    kpi_df['Manufacturing costs'] /
    kpi_df['Revenue generated']
)
kpi_df['quality_cost_exposure'].describe()

count    100.000000
mean       0.029849
std        0.043026
min        0.000067
25%        0.004947
50%        0.014477
75%        0.037660
max        0.258890
Name: quality_cost_exposure, dtype: float64

**Quality Cost Exposure Results**

- On average, 2.98% of revenue is exposed to quality-related costs.
- The top 25% of products exceed 3.77% exposure, indicating significant financial risk.
- The maximum exposure reaches nearly 26% of revenue, highlighting critical products requiring immediate corrective actions.

This KPI connects quality performance directly to financial impact and supports data-driven quality improvement decisions.


##  Power BI Dataset Preparation

This section prepares a clean KPI dataset for visualization and dashboarding in Power BI.
All KPIs are precomputed in Python to ensure consistency and analytical transparency.


In [21]:
powerbi_df = kpi_df[[
    'SKU',
    'Product type',
    'Supplier name',
    'Location',
    'Price',
    'Revenue generated',
    'Number of products sold',
    'Stock levels',
    
    # Inventory & Demand KPIs
    'stock_coverage_ratio',
    'stock_out_flag',
    'demand_pressure_index',
    
    # Lead Time KPIs
    'avg_total_lead_time',
    'mfg_lead_time_share',
    'long_lead_time_flag',
    
    # Cost KPIs
    'cost_per_unit_sold',
    'shipping_cost_intensity',
    'total_cost_per_revenue',
    
    # Quality KPIs
    'Defect rates',
    'quality_cost_exposure',
    'Inspection results'
]]
powerbi_df.to_csv("../Database/supply_chain_kpis.csv", index=False)
